In [32]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import re
import math
import random
from nltk.util import ngrams
from collections import Counter
import nltk
nltk.download('punkt', force=True)
import spacy
nlp = spacy.load("en_core_web_sm")

nltk.data.path.append('.')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# nltk.download('all')

In [4]:
corpus = [
    "<s> Lyn drinks chocolate </s>",
    "<s> She loves chocolate </s>",
    "<s> He drinks tea </s>"
]

In [5]:
vectorizer = CountVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(corpus)

In [6]:
bigrams = vectorizer.get_feature_names_out()
bigrams

array(['drinks chocolate', 'drinks tea', 'he drinks', 'loves chocolate',
       'lyn drinks', 'she loves'], dtype=object)

In [7]:
X.toarray()

array([[1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 1],
       [0, 1, 1, 0, 0, 0]])

In [8]:
def calculate_bigram_probability(word1, word2, vectorizer, X):
    bigrams = vectorizer.get_feature_names_out()
    bigram_counts = X.toarray().sum(axis=0)
    bigram = f"{word1} {word2}"

    if bigram not in bigrams:
        return 0

    bigram_index = list(bigrams).index(bigram)
    bigram_count = bigram_counts[bigram_index]

    word1_count = sum(
        bigram_counts[i]
        for i, bigram in enumerate(bigrams)
        if bigram.startswith(word1)
    )

    return bigram_count / word1_count if word1_count > 0 else 0

In [9]:
print("P(chocolate | drinks):", calculate_bigram_probability("drinks", "chocolate", vectorizer, X))

P(chocolate | drinks): 0.5


<a name='1'></a>
## 1 - Load and Preprocess Data

<a name='1.1'></a>
### 1.1 - Load the Data
You will use twitter data.
Load the data and view the first few sentences by running the next cell.

Notice that data is a long string that contains many many tweets.
Observe that there is a line break "\n" between tweets.

In [10]:
with open("/content/en_US.twitter.txt", 'r') as file:
    data = file.read()

In [11]:
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------


<a name='1.2'></a>
### 1.2 - Pre-process the Data

Preprocess this data with the following steps:

1. Split data into sentences using "\n" as the delimiter.
1. Split each sentence into tokens. Note that in this assignment we use "token" and "words" interchangeably.
1. Assign sentences into train or test sets.
1. Find tokens that appear at least N times in the training data.
1. Replace tokens that appear less than N times by `<unk>`


Note: we omit validation data in this exercise.
- In real applications, we should hold a part of data as a validation set and use it to tune our training.
- We skip this process for simplicity.

<a name='ex-1'></a>
### Exercise 1- split_to_sentences

Split data into sentences.

In [12]:
def split_into_sentence(data):

  sentences = data.split("\n")
  sentences = [s.strip() for s in sentences]
  sentences = [s for s in sentences if len(s) > 0]

  return sentences

In [13]:
x = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""
split_into_sentence(x)

['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

<a name='ex-2'></a>
### Exercise 2 - tokenize_sentences
The next step is to tokenize sentences (split a sentence into a list of words).
- Convert all tokens into lower case so that words which are capitalized (for example, at the start of a sentence) in the original text are treated the same as the lowercase versions of the words.
- Append each tokenized list of words into a list of tokenized sentences.

In [14]:
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        doc = nlp(sentence.lower())
        tokenized_sentences.append([token.text for token in doc])
    return tokenized_sentences

In [15]:
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
print(tokenize_sentences(sentences))

[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green', '.'], ['roses', 'are', 'red', '.']]


<a name='ex-3'></a>
### Exercise 3 - get_tokenized_data


Use the two functions that you have just implemented to get the tokenized data.
- split the data into sentences
- tokenize those sentences

In [16]:
def get_tokenized_data(data):
  sentences = split_into_sentence(data)
  tokenized_sentences = tokenize_sentences(sentences)
  return tokenized_sentences

In [17]:
x = "Sky is blue.\nLeaves are green\nRoses are red."
get_tokenized_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

In [18]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

#### Split into train and test sets

Now run the cell below to split data into training and test sets.

In [19]:
print("{} data are split into {} train and {} test set".format(
    len(tokenized_data), len(train_data), len(test_data)))

print("First training sample:")
print(train_data[0])

print("First test sample")
print(test_data[0])

47961 data are split into 38368 train and 9593 test set
First training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']
First test sample
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']


<a name='ex-4'></a>
### Exercise 4 - count_words

You won't use all the tokens (words) appearing in the data for training.  Instead, you will use the more frequently used words.  
- You will focus on the words that appear at least N times in the data.
- First count how many times each word appears in the data.

You will need a double for-loop, one for sentences and the other for tokens within a sentence.

In [20]:
from collections import Counter

def count_words(tokenized_sentences):
    word_counts = Counter([token for sentence in tokenized_sentences for token in sentence])
    return word_counts

In [21]:
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

Counter({'sky': 1,
         'is': 1,
         'blue': 1,
         '.': 3,
         'leaves': 1,
         'are': 2,
         'green': 1,
         'roses': 1,
         'red': 1})

#### Handling 'Out of Vocabulary' words

If your model is performing autocomplete, but encounters a word that it never saw during training, it won't have an input word to help it determine the next word to suggest. The model will not be able to predict the next word because there are no counts for the current word.
- This 'new' word is called an 'unknown word', or <b>out of vocabulary (OOV)</b> words.
- The percentage of unknown words in the test set is called the <b> OOV </b> rate.

To handle unknown words during prediction, use a special token to represent all unknown words 'unk'.
- Modify the training data so that it has some 'unknown' words to train on.
- Words to convert into "unknown" words are those that do not occur very frequently in the training set.
- Create a list of the most frequent words in the training set, called the <b> closed vocabulary </b>.
- Convert all the other words that are not part of the closed vocabulary to the token 'unk'.




<a name='ex-5'></a>
### Exercise 5 - get_words_with_nplus_frequency

You will now create a function that takes in a text document and a threshold `count_threshold`.
- Any word whose count is greater than or equal to the threshold `count_threshold` is kept in the closed vocabulary.
- Returns the word closed vocabulary list.

In [22]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):

    word_counts = count_words(tokenized_sentences)
    return [word for word, count in word_counts.items() if count >= count_threshold]

In [23]:
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
tmp_closed_vocab = get_words_with_nplus_frequency(tokenized_sentences, count_threshold=2)
print(f"Closed vocabulary:")
print(tmp_closed_vocab)

Closed vocabulary:
['.', 'are']


<a name='ex-6'></a>
### Exercise 6 - replace_oov_words_by_unk

The words that appear `count_threshold` times or more are in the closed vocabulary.
- All other words are regarded as `unknown`.
- Replace words not in the closed vocabulary with the token `<unk>`.

In [24]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):

  vocabulary = set(vocabulary)
  replaced_tokenized_sentences = []

  for sentence in tokenized_sentences:
    replaced_sentence = []

    for token in sentence:

      if token in vocabulary:
        replaced_sentence.append(token)
      else:
        replaced_sentence.append(unknown_token)

    replaced_tokenized_sentences.append(replaced_sentence)

  return replaced_tokenized_sentences

In [25]:
tokenized_sentences = [["dogs", "run"], ["cats", "sleep"]]
vocabulary = ["dogs", "sleep"]
tmp_replaced_tokenized_sentences = replace_oov_words_by_unk(tokenized_sentences, vocabulary)
print(f"Original sentence:")
print(tokenized_sentences)
print(f"tokenized_sentences with less frequent words converted to '<unk>':")
print(tmp_replaced_tokenized_sentences)

Original sentence:
[['dogs', 'run'], ['cats', 'sleep']]
tokenized_sentences with less frequent words converted to '<unk>':
[['dogs', '<unk>'], ['<unk>', 'sleep']]


<a name='ex-7'></a>
### Exercise 7 - preprocess_data

Now we are ready to process our data by combining the functions that you just implemented.

1. Find tokens that appear at least count_threshold times in the training data.
1. Replace tokens that appear less than count_threshold times by "<unk\>" both for training and test data.

In [26]:
def preprocess_data(train_data, test_data, count_threshold, unknown_token="<unk>", get_words_with_nplus_frequency=get_words_with_nplus_frequency, replace_oov_words_by_unk=replace_oov_words_by_unk):

  vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)

  train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary, unknown_token)

  test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary, unknown_token)

  return train_data_replaced, test_data_replaced, vocabulary

In [27]:
tmp_train = [['sky', 'is', 'blue', '.'],
     ['leaves', 'are', 'green']]
tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_data(tmp_train,
                                                           tmp_test,
                                                           count_threshold = 1
                                                          )

print("tmp_train_repl")
print(tmp_train_repl)
print()
print("tmp_test_repl")
print(tmp_test_repl)
print()
print("tmp_vocab")
print(tmp_vocab)

tmp_train_repl
[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]

tmp_test_repl
[['<unk>', 'are', '<unk>', '.']]

tmp_vocab
['sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


#### Preprocess the train and test data
Run the cell below to complete the preprocessing both for training and test sets.

In [28]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data,
                                                                        test_data,
                                                                        minimum_freq)

In [29]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']

First preprocessed test sample:
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']

First 10 vocabulary:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the']

Size of vocabulary: 14841


<a name='2'></a>
## 2 - Develop n-gram based Language Models

In this section, you will develop the n-grams language model.
- Assume the probability of the next word depends only on the previous n-gram.
- The previous n-gram is the series of the previous 'n' words.

The conditional probability for the word at position 't' in the sentence, given that the words preceding it are $w_{t-n}\cdots w_{t-2}, w_{t-1}$ is:

$$ P(w_t | w_{t-n}\dots w_{t-1} ) \tag{1}$$

You can estimate this probability  by counting the occurrences of these series of words in the training data.
- The probability can be estimated as a ratio, where
- The numerator is the number of times word 't' appears after words t-n through t-1 appear in the training data.
- The denominator is the number of times word t-n through t-1 appears in the training data.


$$ \hat{P}(w_t | w_{t-n} \dots w_{t-1}) = \frac{C(w_{t-n}\dots w_{t-1}, w_t)}{C(w_{t-n}\dots w_{t-1})} \tag{2} $$


- The function $C(\cdots)$ denotes the number of occurence of the given sequence.
- $\hat{P}$ means the estimation of $P$.
- Notice that denominator of the equation (2) is the number of occurence of the previous $n$ words, and the numerator is the same sequence followed by the word $w_t$.

Later, you will modify the equation (2) by adding k-smoothing, which avoids errors when any counts are zero.

The equation (2) tells us that to estimate probabilities based on n-grams, you need the counts of n-grams (for denominator) and (n+1)-grams (for numerator).

In [34]:
def count_n_grams(data, n, start_token='<s>', end_token='<e>'):

    n_grams = Counter()

    for sentence in data:
        sentence = [start_token] * (n - 1) + sentence + [end_token]
        n_grams.update(ngrams(sentence, n))

    return dict(n_grams)

In [35]:
bigram_counts = count_n_grams(train_data_processed, n=2)
trigram_counts = count_n_grams(train_data_processed, n=3)

In [36]:
print("Top 10 Bigrams:", sorted(bigram_counts.items(), key=lambda x: x[1], reverse=True)[:10])

print("Top 10 Trigrams:", sorted(trigram_counts.items(), key=lambda x: x[1], reverse=True)[:10])

Top 10 Bigrams: [(('.', '<e>'), 9871), (('!', '<e>'), 7700), (('!', '!'), 5016), (('<s>', 'i'), 4316), (('?', '<e>'), 3041), (('i', "'m"), 2084), (('#', '<unk>'), 1925), (('<unk>', '.'), 1575), (('<unk>', '<e>'), 1409), (('it', "'s"), 1389)]
Top 10 Trigrams: [(('<s>', '<s>', 'i'), 4316), (('!', '!', '!'), 2247), (('!', '!', '<e>'), 1381), (('<s>', '<s>', '<unk>'), 1256), (('<s>', '<s>', '#'), 936), (('<s>', '<s>', 'thanks'), 853), (('<s>', '<s>', 'you'), 790), (('<s>', '<s>', 'the'), 701), (('<s>', 'i', "'m"), 615), (('<s>', '<s>', 'rt'), 605)]


<a name='ex-9'></a>
### Exercise 9 - estimate_probability

Next, estimate the probability of a word given the prior 'n' words using the n-gram counts.

$$ \hat{P}(w_t | w_{t-n} \dots w_{t-1}) = \frac{C(w_{t-n}\dots w_{t-1}, w_t)}{C(w_{t-n}\dots w_{t-1})} \tag{2} $$

This formula doesn't work when a count of an n-gram is zero..
- Suppose we encounter an n-gram that did not occur in the training data.  
- Then, the equation (2) cannot be evaluated (it becomes zero divided by zero).

A way to handle zero counts is to add k-smoothing.  
- K-smoothing adds a positive constant $k$ to each numerator and $k \times |V|$ in the denominator, where $|V|$ is the number of words in the vocabulary.

$$ \hat{P}(w_t | w_{t-n} \dots w_{t-1}) = \frac{C(w_{t-n}\dots w_{t-1}, w_t) + k}{C(w_{t-n}\dots w_{t-1}) + k|V|} \tag{3} $$


For n-grams that have a zero count, the equation (3) becomes $\frac{1}{|V|}$.
- This means that any n-gram with zero count has the same probability of $\frac{1}{|V|}$.

Define a function that computes the probability estimate (3) from n-gram counts and a constant $k$.

- The function takes in a dictionary 'n_gram_counts', where the key is the n-gram and the value is the count of that n-gram.
- The function also takes another dictionary n_plus1_gram_counts, which you'll use to find the count for the previous n-gram plus the current word.

In [37]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts.get(previous_n_gram, 0)

    denominator = previous_n_gram_count + k * vocabulary_size

    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)

    numerator = n_plus1_gram_count + k

    probability = numerator / denominator

    return probability

In [40]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))


In [41]:
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
tmp_prob = estimate_probability("cat", ["a"], unigram_counts, bigram_counts, len(unique_words), k=1)

print(f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob:.4f}")

The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333


#### Estimate probabilities for all words

The function defined below loops over all words in vocabulary to calculate probabilities for all possible words.
- This function is provided for you.

In [42]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, end_token='<e>', unknown_token="<unk>",  k=1.0):

    previous_n_gram = tuple(previous_n_gram)

    vocabulary = vocabulary + [end_token, unknown_token]
    vocabulary_size = len(vocabulary)

    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k=k)

        probabilities[word] = probability

    return probabilities

In [43]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

estimate_probabilities(["a"], unigram_counts, bigram_counts, unique_words, k=1)

{'cat': 0.2727272727272727,
 'i': 0.09090909090909091,
 'this': 0.09090909090909091,
 'a': 0.09090909090909091,
 'like': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'is': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [44]:
trigram_counts = count_n_grams(sentences, 3)
estimate_probabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k=1)

{'cat': 0.1111111111111111,
 'i': 0.2222222222222222,
 'this': 0.2222222222222222,
 'a': 0.1111111111111111,
 'like': 0.1111111111111111,
 'dog': 0.1111111111111111,
 'is': 0.1111111111111111,
 '<e>': 0.1111111111111111,
 '<unk>': 0.1111111111111111}

<a name='3'></a>
## 3 - Perplexity

In this section, you will generate the perplexity score to evaluate your model on the test set.
- You will also use back-off when needed.
- Perplexity is used as an evaluation metric of your language model.
- To calculate the perplexity score of the test set on an n-gram model, use:

$$ PP(W) =\sqrt[N]{ \prod_{t=n+1}^N \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \tag{4}$$

- where $N$ is the length of the sentence.
- $n$ is the number of words in the n-gram (e.g. 2 for a bigram).
- In math, the numbering starts at one and not zero.

In code, array indexing starts at zero, so the code will use ranges for $t$ according to this formula:

$$ PP(W) =\sqrt[N]{ \prod_{t=n}^{N-1} \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \tag{4.1}$$

The higher the probabilities are, the lower the perplexity will be.
- The more the n-grams tell us about the sentence, the lower the perplexity score will be.

<a name='ex-10'></a>
### Exercise  - calculate_perplexity
Compute the perplexity score given an N-gram count matrix and a sentence.

**Note:** For the sake of simplicity, in the code below, `<s>` is included in perplexity score calculation.

In [45]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, start_token='<s>', end_token='<e>', k=1.0):

    n = len(list(n_gram_counts.keys())[0])

    sentence = [start_token] * n + sentence + [end_token]

    sentence = tuple(sentence)

    N = len(sentence)

    product_pi = 1.0

    for t in range(n, N):

        n_gram = sentence[t-n:t]
        word = sentence[t]

        n_gram_count = n_gram_counts.get(n_gram, 0)
        n_plus1_gram = n_gram + (word,)
        n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)

        probability = (n_plus1_gram_count + k) / (n_gram_count + k * vocabulary_size)

        product_pi *= (1 / probability)

    perplexity = product_pi**(1 / N)

    return perplexity

In [46]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexity for first train sample: {perplexity_train:.4f}")

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print(f"Perplexity for test sample: {perplexity_test:.4f}")

Perplexity for first train sample: 2.6889
Perplexity for test sample: 3.8027


<a name='4'></a>
## 4 - Build an Auto-complete System

In this section, you will combine the language models developed so far to implement an auto-complete system.


<a name='ex-11'></a>
### Exercise - suggest_a_word
Compute probabilities for all possible next words and suggest the most likely one.
- This function also take an optional argument `start_with`, which specifies the first few letters of the next words.

In [47]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):

    n = len(list(n_gram_counts.keys())[0])

    previous_n_gram = previous_tokens[-n:]

    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    suggestion = None

    max_prob = 0

    for word, prob in probabilities.items():

        if start_with:

            if not word.startswith(start_with):

                continue

        if prob > max_prob:

            suggestion = word
            max_prob = prob
    return suggestion, max_prob

In [48]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")

The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.0909


#### Get multiple suggestions

The function defined below loops over various n-gram models to get multiple suggestions.

In [52]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]

        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

#### Suggest multiple words using n-grams of varying length

Congratulations!  You have developed all building blocks for implementing your own auto-complete systems.

Let's see this with n-grams of varying lengths (unigrams, bigrams, trigrams, 4-grams...6-grams).

In [53]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [54]:
previous_tokens = ["i", "want", "to", "go"]
tmp_suggest1 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest1)

The previous words are ['i', 'want', 'to', 'go'], the suggestions are:


[('to', 0.014029180695847363),
 ('to', 0.004691733298090266),
 ('to', 0.0009411764705882353),
 ('to', 0.0004039044092898014)]

In [55]:
previous_tokens = ["hey", "how", "are"]
tmp_suggest2 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest2)

The previous words are ['hey', 'how', 'are'], the suggestions are:


[('you', 0.02334054423317773),
 ('you', 0.003554661301140174),
 ('you', 0.00013473457289140393),
 ('i', 6.737182510274203e-05)]